In [19]:
from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# 감성사전 데이터 가져와서 점수만 뽑기

In [2]:
temp = pd.read_csv('news_with_sent_score.csv')
temp.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
sent_score = temp['sent_score']

In [4]:
sent_score

0     -6.268386e+07
1     -6.255832e+07
2     -6.957441e+07
3     -7.305733e+07
4     -9.356633e+07
           ...     
115   -5.824726e+07
116   -6.441351e+07
117   -3.751069e+07
118   -3.147741e+07
119   -3.986893e+07
Name: sent_score, Length: 120, dtype: float64

In [5]:
del temp

# 기존의 데이터 가져와서 합치기

In [6]:
df = pd.read_csv('분석용데이터_utf.csv')

In [7]:
#Date를 index로 넣어줌
df.index = df['Date']
df.drop(['Date'], axis = 1, inplace = True)

In [8]:
df

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,spi
Date,,,,,,,
201101월,1676.4,4.52,2.75,3.4,2069.73,1119.25,81.2
201102월,1674.4,4.72,2.75,3.9,1939.30,1124.65,82.5
201103월,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.4
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00,83.9
201105월,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.1
...,...,...,...,...,...,...,...
202008월,3100.4,2.19,0.50,0.7,2326.17,1187.94,113.2
202009월,3115.2,2.24,0.50,1.0,2327.89,1164.65,114.2
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49,115.8


In [9]:
df.spi.head(6)

Date
201101월    81.2
201102월    82.5
201103월    83.4
201104월    83.9
201105월    84.1
201106월    84.3
Name: spi, dtype: float64

In [10]:
sent_score

0     -6.268386e+07
1     -6.255832e+07
2     -6.957441e+07
3     -7.305733e+07
4     -9.356633e+07
           ...     
115   -5.824726e+07
116   -6.441351e+07
117   -3.751069e+07
118   -3.147741e+07
119   -3.986893e+07
Name: sent_score, Length: 120, dtype: float64

In [11]:
sent_score.index = df.index

In [12]:
df = pd.concat([sent_score, df], axis = 1)

In [13]:
df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,spi
Date,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,81.2
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,82.5
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.4
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,83.9
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.1
...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,113.2
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,114.2
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,115.8


# Preprocessing

+ spi를 일단 하나씩 sliding 해야 됨. 타겟에 들어갈 것은 다음 달 spi니까.
    - 예를 들어 2011년 1월 index에 들어가야 하는 next_spi 값은 2011년 2월의 spi가 들어가야 함.


+ 전달의 spi와 차가 음수 -> 0, 차가 0 이상 -> 1 (차이없는 것은 상방으로 넣기로 했으므로)

## spi sliding

In [14]:
df['next_spi'] = df['spi'].shift(-1)
df['next_spi'][-1] = 125.4

In [15]:
df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,spi,next_spi
Date,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,81.2,82.5
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,82.5,83.4
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.4,83.9
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,83.9,84.1
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.1,84.3
...,...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,113.2,114.2
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,114.2,115.8
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,115.8,118.6


In [16]:
df.drop(columns = 'spi', inplace = True)

In [17]:
df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,next_spi
Date,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,82.5
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,83.4
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.9
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,84.1
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.3
...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,114.2
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,115.8
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,118.6


# SVM Regressor

## 감성 지수 반영 X

In [16]:
X = df.iloc[:,1:]
y = X.pop('next_spi')

In [38]:
#90개월이 아닌 다른 경우의 수에도 적용할 수 있도록...
#X, y, 몇 개월을 검토할지 입력
#X_train, y_train은 그전것 전부, X_test은 그 다음 한 개월치만...  y_test은 굳이 안함. 어차피 나중에 실제 y에서 인덱싱하면되자나

def split_train_test(X, y , num_of_train):
    """
    ex) split_train_test(X, y, 90) 
    
    => X_train 은 90개의 앞의 월별 데이터, X_test는 다음 91개째 하나의 월별 데이터를 반환
    """
    X_train = X.iloc[:num_of_train]
    X_test = pd.DataFrame(X.iloc[num_of_train]).T
    y_train = y.iloc[:num_of_train]
    
    return X_train, X_test, y_train

def train_predict(X, y, num_of_train):
    """
    y_pred = [90개월치 훈련 91개월차 예측값, 91개월치 훈련 92개월차 예측값, ...119개월 훈련 120개월차 예측값] 을 만들기 위해
    먼저 y_pred = [] 초기화
    
    y_test는 결과(confusion matrix) 비교 위해 list형태로 바꿔줌.
    num_of_train 입력해주면 그 시점부터 마지막 하나 전 월까지 훈련시키고 테스트하고를 반복해서 y_pred에 그때마다 넣어주고
    한 train data 사용하면 메모리 정리를 위해 지워주고 다음 for 문에서 재할당해줌.
    
    예를 들어, i가 90이면
    i = 90 ->train : 0~89 데이터 => 훈련해서 90개차 test data 한 줄 넣고 예측 결과 뽑아서 그걸 y_pred에 추가 -> data delete
    i = 91 ->train : 0~90 데이터 => 훈련해서 91개차 test data 한 줄 넣고 예측 결과 뽑아서 그걸 y_pred에 추가 -> data delete
    ... 반복...
    
    """
    y_pred = []
    y_test = y[num_of_train:].tolist()
    for i in range(num_of_train, X.shape[0]):
        # 1 ~ i, i+1 의 데이터 생성
        X_train, X_test, y_train = split_train_test(X, y, i)
        
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        # default parameter 
        svr = SVR(kernel = 'sigmoid')
        svr.fit(X_train, y_train)
        pred = svr.predict(X_test)
        y_pred.append(pred[0])
        del X_train, X_test, y_train #메모리 삭제

    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))
    print(mean_squared_error(y_test, y_pred, squared = False)) # squared = False면 RMSE, True면 MSE 출력
    
    # RMSE = Root Mean Squared Error : RMSE가 낮을수록 정확도가 높음을 의미
    
#    print("auc: ", roc_auc_score(y_test,y_pred))

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- linear : 1.6551
- poly : 8.1534
- rbf : 10.0820
- sigmoid : 7.8774

In [39]:
train_predict(X, y, 90)

7.877466951423792


In [40]:
del X,y

## 감성 지수 반영

In [41]:
X = df.copy()
y = X.pop('next_spi')

In [51]:
#90개월이 아닌 다른 경우의 수에도 적용할 수 있도록...
#X, y, 몇 개월을 검토할지 입력
#X_train, y_train은 그전것 전부, X_test은 그 다음 한 개월치만...  y_test은 굳이 안함. 어차피 나중에 실제 y에서 인덱싱하면되자나

def split_train_test(X, y , num_of_train):
    """
    ex) split_train_test(X, y, 90) 
    
    => X_train 은 90개의 앞의 월별 데이터, X_test는 다음 91개째 하나의 월별 데이터를 반환
    """
    X_train = X.iloc[:num_of_train]
    X_test = pd.DataFrame(X.iloc[num_of_train]).T
    y_train = y.iloc[:num_of_train]
    
    return X_train, X_test, y_train

def train_predict(X, y, num_of_train):
    """
    y_pred = [90개월치 훈련 91개월차 예측값, 91개월치 훈련 92개월차 예측값, ...119개월 훈련 120개월차 예측값] 을 만들기 위해
    먼저 y_pred = [] 초기화
    
    y_test는 결과(confusion matrix) 비교 위해 list형태로 바꿔줌.
    num_of_train 입력해주면 그 시점부터 마지막 하나 전 월까지 훈련시키고 테스트하고를 반복해서 y_pred에 그때마다 넣어주고
    한 train data 사용하면 메모리 정리를 위해 지워주고 다음 for 문에서 재할당해줌.
    
    예를 들어, i가 90이면
    i = 90 ->train : 0~89 데이터 => 훈련해서 90개차 test data 한 줄 넣고 예측 결과 뽑아서 그걸 y_pred에 추가 -> data delete
    i = 91 ->train : 0~90 데이터 => 훈련해서 91개차 test data 한 줄 넣고 예측 결과 뽑아서 그걸 y_pred에 추가 -> data delete
    ... 반복...
    
    """
    y_pred = []
    y_test = y[num_of_train:].tolist()
    for i in range(num_of_train, X.shape[0]):
        # 1 ~ i, i+1 의 데이터 생성
        X_train, X_test, y_train = split_train_test(X, y, i)
        
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        # default parameter 
        svr = SVR(kernel = 'sigmoid')
        svr.fit(X_train, y_train)
        pred = svr.predict(X_test)
        y_pred.append(pred[0])
        del X_train, X_test, y_train #메모리 삭제

    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))
    print(mean_squared_error(y_test, y_pred, squared = False)) # squared = False면 RMSE, True면 MSE 출력
    
    # RMSE = Root Mean Squared Error : RMSE가 낮을수록 정확도가 높음을 의미
    
#    print("auc: ", roc_auc_score(y_test,y_pred))

In [52]:
train_predict(X, y, 90)

7.060026441837677


**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- linear : 1.6551 => 1.7038
- poly : 8.1534 => 6.9597
- rbf : 10.0820 => 10.2330
- sigmoid : 7.8774 => 7.0600